In [8]:
from entropy_compressor import ZCompressor
from quantizator import Quantizator
from wavelet_compressor import WaveletCompressor

import os
import numpy as np
from typing import Dict, List
from pathlib import Path
from pprint import pprint

COMPRESS_PATH = 'compressed_low'
DECOMPRESS_PATH = 'decompressed'
IMAGE_NAME = 'low.png'
MESH_KEYS = ['faces', 'neighbour', 'owner', 'points']
PTURHO_KEYS = ['p', 'T', 'U', 'rho']

class Decompressor:
    def __init__(self):
        if not os.path.exists(DECOMPRESS_PATH):
            os.mkdir(DECOMPRESS_PATH)
        self.entropy_compressor = ZCompressor()
        self.quantizator = Quantizator()
        self.wavelet_compressor = WaveletCompressor()
    
    def total_decompress(self, compressed_data: str, q_step: np.float64, min_val: np.float64) -> np.ndarray:
        z_decoded = self.entropy_compressor.decompress(bytes.fromhex(compressed_data))
        print('z decoded')
        dequantized = self.quantizator.dequantize(z_decoded, q_step, min_val)
        print('dequantized')
        decompressed = self.wavelet_compressor.inverse_wavelet_transform(dequantized)

        return decompressed

    def load_to_file(self, data: List[np.ndarray | np.float64], file_path: str, header_data: dict[str]) -> None:
        z_compressed, q_step, min_val = data
        decompressed = self.total_decompress(z_compressed, q_step, min_val)
        
    def read_mesh_file(self, file_path: str) -> Dict[str, List[np.ndarray | np.float64]]:
        data = {}
        for key in MESH_KEYS:
            data[key] = None
        i = 0
        with open(file_path, mode='r') as f:
            while True:
                # Read the first line
                quantilization_keys = f.readline().strip()
                # Read the second line
                huffman_compressed = f.readline().strip()

                # Check if either line is empty, indicating the end of the file
                if not quantilization_keys or not huffman_compressed:
                    break
                
                q_step, min_val = map(float, quantilization_keys.split())
                data[MESH_KEYS[i]] = [huffman_compressed, q_step, min_val]
                i += 1
        return data
    
    def read_pturho_file(self, file_path: str) -> Dict[str, List[np.ndarray | np.float64]]:
        data = {}
        i = 0
        with open(file_path, mode='r') as f:
            times = f.readline().strip().split(';')
            for key in times:
                data[key] = None
            while True:
                quantilization_keys_p = f.readline().strip()
                huffman_compressed_p = f.readline().strip()

                quantilization_keys_t = f.readline().strip()
                huffman_compressed_t = f.readline().strip()

                quantilization_keys_u = f.readline().strip()
                huffman_compressed_u = f.readline().strip()

                quantilization_keys_rho = f.readline().strip()
                huffman_compressed_rho = f.readline().strip()

                # Check if either line is empty, indicating the end of the file
                if not quantilization_keys_p or not huffman_compressed_p:
                    break
                
                q_step_p, min_val_p = map(float, quantilization_keys_p.split())
                q_step_t, min_val_t = map(float, quantilization_keys_t.split())
                q_step_u, min_val_u = map(float, quantilization_keys_u.split())
                q_step_rho, min_val_rho = map(float, quantilization_keys_rho.split())
                data[times[i]] = [
                    [huffman_compressed_p, q_step_p, min_val_p],
                    [huffman_compressed_t, q_step_t, min_val_t],
                    [huffman_compressed_u, q_step_u, min_val_u],
                    [huffman_compressed_rho, q_step_rho, min_val_rho],
                ]
                i += 1
        return data

    def copy_back_not_compressed_piece(self, file_path: str, destination_path: str) -> None:
        # shutil
        pass

In [12]:
decompressor = Decompressor()
mesh_data = decompressor.read_mesh_file('compressed/mesh')
decompressor.total_decompress(*mesh_data['faces'])

huffman decoded
dequantized


ValueError: Unexpected detail coefficient type: <class 'numpy.float64'>. Detail coefficients must be a dictionary of arrays as returned by wavedecn. If you are using pywt.array_to_coeffs or pywt.unravel_coeffs, please specify output_format='wavedecn'